In [1]:
import numpy as np
import pandas as pd
import BasicModel as bm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.stats.diagnostic import acorr_ljungbox as lb_test

In [2]:
#获取三个市场的基本数据
fc = bm.basicmodel()
fc.ret(start='2011-08-01',end='2015-08-01')
fc.AR_GARCH(model_lags=[0,[2],2])

Iteration:      1,   Func. Count:      5,   Neg. LLF: -5403.072865708344
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -5403.072866035264
            Iterations: 1
            Function evaluations: 5
            Gradient evaluations: 1
Iteration:      1,   Func. Count:      6,   Neg. LLF: -4953.184909297267
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -4953.184906614894
            Iterations: 1
            Function evaluations: 6
            Gradient evaluations: 1
Iteration:      1,   Func. Count:      7,   Neg. LLF: -5118.994567691181
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -5118.994569771838
            Iterations: 1
            Function evaluations: 7
            Gradient evaluations: 1


c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 6.055e-07. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.731e-06. Parameter
estimation work better when this value is between 1 and 1000. T

In [16]:
model = VAR(fc.std_index[['CNY','CNH']])
aic = {}
bic = {}
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    result = model.fit(i)
    aic[i] = result.aic
    bic[i] = result.bic

c:\Users\86177\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [4]:
def grangers_causation_matrix(data,maxlag,test='ssr_chi2test', verbose=False):    
    #maxlag可以是最大值，也可以是所有需要测试的阶数的一个迭代器
    if type(maxlag) == int:
        alltestlag = [i+1 for i in range(maxlag)]
    else:
        alltestlag = maxlag
    #生成一个pvalue矩阵
    variables = data.columns
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i][0][test][1],4) for i in alltestlag]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_cause' for var in variables]
    df.index = [var + '_result' for var in variables]
    return df

In [31]:
fc = bm.basicmodel()
fc.ret(start='2011-08-01',end='2015-08-01')
fc.AR_GARCH(model_lags=[0,[1,2],[2,7]])

Iteration:      1,   Func. Count:      5,   Neg. LLF: -5403.072865708344
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -5403.072866035264
            Iterations: 1
            Function evaluations: 5
            Gradient evaluations: 1
Iteration:      1,   Func. Count:      7,   Neg. LLF: -4932.184236750706
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -4932.184233199658
            Iterations: 1
            Function evaluations: 7
            Gradient evaluations: 1
Iteration:      1,   Func. Count:      7,   Neg. LLF: -5097.330415072179
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -5097.330416841047
            Iterations: 1
            Function evaluations: 7
            Gradient evaluations: 1


c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 6.055e-07. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.657e-06. Parameter
estimation work better when this value is between 1 and 1000. T

In [32]:
grangers_causation_matrix(fc.std_index,maxlag=7)

,CNY_cause,CNH_cause,NDF_cause
CNY_result,1.0000,0.5309,0.2223
CNH_result,0.0445,1.0000,0.0001
NDF_result,0.1765,0.1952,1.0000


In [39]:
fc = bm.basicmodel()
fc.ret(start='2015-09-01',end='2017-05-01')
fc.AR_GARCH(model_lags=[1,1,0])

Iteration:      1,   Func. Count:      7,   Neg. LLF: 31148381724423.766
Iteration:      2,   Func. Count:     21,   Neg. LLF: 1050599397.9020545
Iteration:      3,   Func. Count:     35,   Neg. LLF: 11992934044.223831
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1706.0838338725807
            Iterations: 5
            Function evaluations: 43
            Gradient evaluations: 3
Iteration:      1,   Func. Count:      7,   Neg. LLF: 433086772498884.1
Iteration:      2,   Func. Count:     21,   Neg. LLF: 1.772361153983856e+16
Iteration:      3,   Func. Count:     37,   Neg. LLF: 9639269056963.158
Iteration:      4,   Func. Count:     53,   Neg. LLF: 442425516581030.25
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1663.6933226581182
            Iterations: 5
            Function evaluations: 61
            Gradient evaluations: 4
Iteration:      1,   Func. Count:      6,   Neg. LLF: 71349161020.3805

c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.251e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 6.478e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: Da

In [40]:
grangers_causation_matrix(fc.std_index,maxlag=7)#.iloc[2,1]

,CNY_cause,CNH_cause,NDF_cause
CNY_result,1.0000,0.00,0.0
CNH_result,0.2560,1.00,0.0
NDF_result,0.1373,0.15,1.0


In [41]:
fc = bm.basicmodel()
fc.ret(start='2017-06-01',end='2023-11-01')
fc.AR_GARCH(model_lags=[1,[6],[1,3]])

Iteration:      1,   Func. Count:      7,   Neg. LLF: 9.060052652323613e+22
Iteration:      2,   Func. Count:     22,   Neg. LLF: 3.7516722952891547e+30
Iteration:      3,   Func. Count:     38,   Neg. LLF: 1.8034459481957293e+24
Iteration:      4,   Func. Count:     51,   Neg. LLF: -5620.600509630422
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -5620.600488599576
            Iterations: 6
            Function evaluations: 51
            Gradient evaluations: 4
Iteration:      1,   Func. Count:      6,   Neg. LLF: -6165.621183231791
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -6165.621177918374
            Iterations: 1
            Function evaluations: 6
            Gradient evaluations: 1
Iteration:      1,   Func. Count:      8,   Neg. LLF: 1356905605260.075
Iteration:      2,   Func. Count:     23,   Neg. LLF: -6131.970028218662
Inequality constraints incompatible    (Exit mode 4)
            C

c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 7.095e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 1.095e-05. Parameter
estimation work better when this value is between 1 and 1000. T

In [42]:
grangers_causation_matrix(fc.std_index,maxlag=7)

,CNY_cause,CNH_cause,NDF_cause
CNY_result,1.0000,0.000,0.0
CNH_result,0.0134,1.000,0.0
NDF_result,0.6043,0.225,1.0
